### 1. importing libs

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 2. Data Preprocessing

In [ ]:
df = pd.read_csv('insurance_claims.csv')

#### 2.1 understanding the data  